<a href="https://colab.research.google.com/github/Utree/deeplearning2_colab_log/blob/master/3%E7%AB%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 単語の分散表現 〜推論ベースの手法word2vec〜
カウントベースの手法では、周囲の単語の頻度によって、共起行列を作り、その行列に対して、SVDを適応することで、密なベクトル単語の分散表現を獲得しました。

しかし、カウントベースの手法には問題点があり、大規模なコーパスを扱う場合、巨大な行列を作ることになります。しかし、巨大な行列に対して、SVDを行うことは現実的ではありません。

## 推論ベースの手法

コンテキストが与えられたときに、どのような単語が出現するのかを推論する手法

推論問題を繰り返し解き、出現パーターンを学習する

モデルにコンテキストを入力として与えると、モデルは各単語の出現確率を出力する

one-hot表現で単語をベクトル表現で表すと

ニューラルネットワークを構成する様々な「レイヤ」によって処理することができる

In [1]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]]) # 入力
W = np.random.randn(7, 3) # 重み
# バイアスを用いいない全結合層のニューラルネット = 「行列の積」
h = np.dot(c, W) # 中間ノード
print(h)

# one-hotベクトルの積では、該当する重みの行ベクトルを抜き出すことに相当する

[[ 0.31117849 -1.89414643 -1.46252998]]


In [0]:
import numpy as np

In [0]:
class MatMul:
  '''
    MatMul(行列の積)レイヤ
  '''
  def __init__(self, W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.x = None
    
  def forward(self, x):
    W, = self.params
    out = np.dot(x, W)
    self.x = x
    return out
  
  def backward(self, dout):
    W, = self.params
    dx = np.dot(dout, W.T)
    dW = np.dot(self.x.T, dout)
    self.grads[0][...] = dW
    return dx

In [5]:
# MatMulレイヤで実装
c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-2.33967714  0.1365977  -1.12722566]]


word2vecでは、ニューラルネットワークのモデルにcontinuous bag-of-words(CBOW)を用いる

## CBOWのデータ処理構造

入力層でone-hot表現のベクトル受け取り、中間層で全結合による変換後の値の平均値を出し、出力層で、各単語のスコアを算出する

そのスコアにSoftmax関数を適応すると、確率が得られる

そして、この確率には、単語の意味もエンコードされている

In [8]:
# サンプルのコンテキストデータ
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 重みの初期化
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# レイヤの生成
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 順伝播
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[ 0.23199029  0.48978974 -0.8598195   0.47971249  1.20140243 -0.8945531
   0.244637  ]]
